In [1]:
import tensorflow as tf
from pathlib import Path
import json
import numpy as np
import requests
import os

2024-09-26 10:55:58.109333: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-26 10:55:58.113616: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-26 10:55:58.127769: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-26 10:55:58.148135: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-26 10:55:58.154357: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-26 10:55:58.168389: I tensorflow/core/platform/cpu_feature_gu

In [2]:
model = tf.keras.models.load_model("model/mnist_model_cnn.keras")

In [3]:
model_name = "my_mnist_model"
model_version = "0001"
model_path = Path(model_name) / model_version
model.export(model_path)

INFO:tensorflow:Assets written to: my_mnist_model/0001/assets


INFO:tensorflow:Assets written to: my_mnist_model/0001/assets


Saved artifact at 'my_mnist_model/0001'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 28, 28, 1), dtype=tf.float32, name='input_layer')
Output Type:
  TensorSpec(shape=(None, 10), dtype=tf.float32, name=None)
Captures:
  140465119516224: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140465119521152: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140465119692400: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140465119697328: TensorSpec(shape=(), dtype=tf.resource, name=None)


In [4]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()

In [5]:
X_new = np.expand_dims(X_test[:3], axis=-1)/255. # pretend we have 3 new digit images to classify
request_json = json.dumps({
    "signature_name": "serving_default",
    "instances": X_new.tolist(),
})

In [6]:
os.environ["MODEL_DIR"] = str(model_path.parent.absolute())

In [7]:
%%bash --bg
tensorflow_model_server \
--port=8500 \
--rest_api_port=8501 \
--model_name=my_mnist_model \
--model_base_path="${MODEL_DIR}" >my_server.log 2>&1

### Using REST API

In [9]:
server_url = "http://localhost:8501/v1/models/my_mnist_model:predict"
response = requests.post(server_url, data=request_json)
response.raise_for_status() # raise an exception in case of error
response = response.json()

In [10]:
y_proba = np.array(response["predictions"])
y_proba.round(2)

array([[0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 1.  , 0.  , 0.  ],
       [0.  , 0.  , 1.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.99, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ]])

In [11]:
y_proba.argmax(axis=1)

array([7, 2, 1])

In [12]:
y_test[:3]

array([7, 2, 1], dtype=uint8)

### Using gRPC

In [16]:
from tensorflow_serving.apis.predict_pb2 import PredictRequest
request = PredictRequest()
request.model_spec.name = model_name
request.model_spec.signature_name = "serving_default"
input_name = model.inputs[0].name
request.inputs[input_name].CopyFrom(tf.make_tensor_proto(X_new, dtype="float32"))

In [17]:
import grpc
from tensorflow_serving.apis import prediction_service_pb2_grpc
channel = grpc.insecure_channel('localhost:8500')
predict_service = prediction_service_pb2_grpc.PredictionServiceStub(channel)
response = predict_service.Predict(request, timeout=10.0)

In [18]:
response.outputs

{'output_0': dtype: DT_FLOAT
tensor_shape {
  dim {
    size: 3
  }
  dim {
    size: 10
  }
}
float_val: 1.15736533e-07
float_val: 4.0681936e-08
float_val: 1.35461705e-05
float_val: 6.05343157e-05
float_val: 1.3926324e-10
float_val: 1.60982594e-07
float_val: 1.73971879e-11
float_val: 0.99991715
float_val: 8.18322192e-08
float_val: 8.22852508e-06
float_val: 1.29933596e-06
float_val: 0.000251691235
float_val: 0.999696136
float_val: 8.13125382e-07
float_val: 8.18114586e-12
float_val: 2.183703e-08
float_val: 1.7922817e-08
float_val: 2.05343198e-07
float_val: 4.97845394e-05
float_val: 1.43922027e-10
float_val: 8.15870681e-06
float_val: 0.992999911
float_val: 7.5164935e-05
float_val: 3.6409383e-05
float_val: 0.00095823
float_val: 6.07134207e-05
float_val: 0.000982121
float_val: 0.00106801733
float_val: 0.00376288965
float_val: 4.84777665e-05
}

In [19]:
model.outputs[0].name

'keras_tensor_9'

In [20]:
model.outputs[0].name in response.outputs

False

In [21]:
# output_name = model.outputs[0].name
# outputs_proto = response.outputs[output_name]
outputs_proto = response.outputs["output_0"]
y_proba = tf.make_ndarray(outputs_proto)

In [22]:
y_proba.argmax(axis=1)

array([7, 2, 1])

In [23]:
y_test[:3]

array([7, 2, 1], dtype=uint8)

### Deploying a new model version

In [ ]:
# treating previous model as a new version for testing instead of training a new one
import copy
model2 = copy.deepcopy(model)
model_version = "0002"
model_path = Path(model_name) / model_version
model2.export(model_path)

INFO:tensorflow:Assets written to: my_mnist_model/0002/assets


INFO:tensorflow:Assets written to: my_mnist_model/0002/assets


Saved artifact at 'my_mnist_model/0002'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 28, 28, 1), dtype=tf.float32, name='input_layer')
Output Type:
  TensorSpec(shape=(None, 10), dtype=tf.float32, name=None)
Captures:
  140280416220992: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140280416733488: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140280416745808: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140280416742464: TensorSpec(shape=(), dtype=tf.resource, name=None)


At regular intervals (the delay is configurable), TF Serving checks the model directory
for new model versions. If it finds one, it automatically handles the transition grace‐
fully: by default, it answers pending requests (if any) with the previous model version,
while handling new requests with the new version. As soon as every pending request
has been answered, the previous model version is unloaded. You can see this at work
in the TF Serving logs (in *my_server.log*):
```log
[...]
Reading SavedModel from: /models/my_mnist_model/0002
Reading meta graph with tags { serve }
[...]
Successfully loaded servable version {name: my_mnist_model version: 2}
Quiescing servable version {name: my_mnist_model version: 1}
Done quiescing servable version {name: my_mnist_model version: 1}
Unloading servable version {name: my_mnist_model version: 1}
```
>If the SavedModel contains some example instances in the assets/
extra directory, you can configure TF Serving to run the new
model on these instances before starting to use it to serve requests.
This is called model warmup: it will ensure that everything is prop‐
erly loaded, avoiding long response times for the first requests.

This approach offers a smooth transition, but it may use too much RAM—especially
GPU RAM, which is generally the most limited. In this case, you can configure TF
Serving so that it handles all pending requests with the previous model version and
unloads it before loading and using the new model version. This configuration will
avoid having two model versions loaded at the same time, but the service will be
unavailable for a short period.

As you can see, TF Serving makes it straightforward to deploy new models. More‐
over, if you discover that version 2 does not work as well as you expected, then rolling
back to version 1 is as simple as removing the my_mnist_model/0002 directory.

>Another great feature of TF Serving is its automatic batching capa‐
bility, which you can activate using the --enable_batching option
upon startup. When TF Serving receives multiple requests within
a short period of time (the delay is configurable), it will automat‐
ically batch them together before using the model. This offers a
significant performance boost by leveraging the power of the GPU.
Once the model returns the predictions, TF Serving dispatches
each prediction to the right client. You can trade a bit of latency
for a greater throughput by increasing the batching delay (see the
--batching_parameters_file option).

If you expect to get many queries per second, you will want to deploy TF Serving
on multiple servers and load-balance the queries (see Figure 19-2). This will require
deploying and managing many TF Serving containers across these servers. One way
to handle that is to use a tool such as Kubernetes, which is an open source system
for simplifying container orchestration across many servers. If you do not want to
purchase, maintain, and upgrade all the hardware infrastructure, you will want to
use virtual machines on a cloud platform such as Amazon AWS, Microsoft Azure,
Google Cloud Platform, IBM Cloud, Alibaba Cloud, Oracle Cloud, or some other
platform as a service (PaaS) offering. Managing all the virtual machines, handling
container orchestration (even with the help of Kubernetes), taking care of TF Serving
configuration, tuning and monitoring—all of this can be a full-time job. Fortunately,
some service providers can take care of all this for you. In this chapter we will
use Vertex AI: it’s the only platform with TPUs today; it supports TensorFlow 2,
Scikit-Learn, and XGBoost; and it offers a nice suite of AI services. There are several
other providers in this space that are capable of serving TensorFlow models as well,
though, such as Amazon AWS SageMaker and Microsoft AI Platform, so make sure
to check them out too.

![image.png](assets/images/load_balancing.png)